## Bert 카테고리 분류 테스트

In [1]:
pip install transformers torch #라이브러리 설치

## Fine-tuning 전 Bert 실행

## Fine-tuning

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split

# CSV 파일 읽기
file_path = "dataset_news_data_detailed.csv"  # CSV 파일 경로
df = pd.read_csv(file_path)

dataset = df[["카테고리", "뉴스 제목", "뉴스 내용"]]
#print(dataset.info())

# 3. 입력(X)와 출력(y) 데이터 분리
x = df[["뉴스 제목", "뉴스 내용"]]  # 입력 데이터
y = df["카테고리"]  # 출력 데이터 (레이블)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [26]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from sklearn.preprocessing import LabelEncoder

# 1. BERT 토크나이저 및 모델 로드
model_name = "bert-base-multilingual-cased"  # 멀티랭귀지 BERT 모델
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=8  # 분류할 카테고리 개수
)

# 2. 카테고리 이름 정의
categories = [
    "금융", "증권", "산업/재계", "중기/벤처", "부동산",
    "글로벌 경제", "생활경제", "경제 일반"
]

# 데이터 준비
def preprocess_data(x, y, tokenizer, max_len=128):
    # 결측값을 빈 문자열로 대체
    x["뉴스 제목"] = x["뉴스 제목"].fillna("")
    x["뉴스 내용"] = x["뉴스 내용"].fillna("")
    
    # 뉴스 제목과 내용을 합쳐서 리스트 형태로 변환
    combined_texts = (x["뉴스 제목"] + " " + x["뉴스 내용"]).tolist()
    inputs = tokenizer(
        combined_texts,  # 뉴스 제목과 내용을 합침
        max_length=max_len,
        padding='max_length',  # 패딩 추가
        truncation=True,       # 최대 길이 초과 시 자름
        return_tensors="pt"    # PyTorch 텐서 형식 반환
    )
    labels = y.tolist()  # 레이블을 리스트로 변환
    return inputs, labels

# Train 데이터셋 처리
train_inputs, train_labels = preprocess_data(x_train, y_train, tokenizer)

# Test 데이터셋 처리
test_inputs, test_labels = preprocess_data(x_test, y_test, tokenizer)

# LabelEncoder로 문자열 레이블을 정수로 변환
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)  # 문자열 -> 정수
test_labels = label_encoder.transform(test_labels)  # 문자열 -> 정수

#TensorDataset 생성
train_dataset = TensorDataset(
    train_inputs['input_ids'],
    train_inputs['attention_mask'],
    torch.tensor(train_labels)  # 정수형 레이블
)
test_dataset = TensorDataset(
    test_inputs['input_ids'],
    test_inputs['attention_mask'],
    torch.tensor(test_labels)  # 정수형 레이블
)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# BERT 모델 로드 (분류용으로 사전학습된 모델)
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=len(y_train.unique())  # 출력 레이블 수
)

# Optimizer 설정
optimizer = AdamW(model.parameters(), lr=2e-5)

# 손실 함수 설정
loss_fn = CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [28]:
from tqdm import tqdm

# 학습 루프
epochs = 3
for epoch in range(epochs):
    model.train()  # 학습 모드 활성화
    total_loss = 0

    for batch in tqdm(train_loader):
        # 데이터에서 배치 추출
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]

        # Forward 및 Backward 패스
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

  2%|█▏                                                                            | 6/392 [51:53<55:38:32, 518.94s/it]


KeyboardInterrupt: 

### 모델 평가

In [ ]:
# 평가 루프
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).numpy())
        true_labels.extend(labels.numpy())

# 성능 평가
from sklearn.metrics import classification_report
print(classification_report(true_labels, predictions, target_names=label_encoder.classes_))
